In [ ]:
!pip install confluent-kafka

In [1]:
from dataclasses import dataclass
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka.admin import ConfigResource, ResourceType
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException

In [2]:
bootstrap_servers = 'kafka1:19091,kafka1:19092,kafka1:19093'
topic_name = 'hyunsoo'
group_id = "G1"
admin_client = AdminClient({'bootstrap.servers' : bootstrap_servers})

In [3]:
@dataclass
class ConfluentKafka:
    bootstrap_servers: str

    def __post_init__(self):
        self.admin_client =  AdminClient({'bootstrap.servers' : self.bootstrap_servers})
        self.producer = Producer({'bootstrap.servers': self.bootstrap_servers})

    def create_topic(self, topic_name, num_partitions, replication_factor):
        new_topic = NewTopic(topic_name, num_partitions, replication_factor)
        self.admin_client.create_topics([new_topic])

    def list_topic(self):
        topic_meta = self.admin_client.list_topics(timeout=10)
        self.topic_list = topic_meta.topics
        for topic in self.topic_list:
            print(topic)
            
    def modify_partition(self, topic_name, partition_count):
        new_partitions = NewPartitions(topic_name, new_total_count = partition_count)
        self.admin_client.create_partitions([new_partitions])
    
    def describe_topic(self, topic_name):
        topic_metadata = self.topic_list[topic_name]
        return {
            'partitions': len(topic_metadata.partitions),
            'replication_factor': len(next(iter(topic_metadata.partitions.values())).replicas),
            'config': admin_client.describe_configs([ConfigResource(ResourceType.TOPIC, topic_name)])[ConfigResource(ResourceType.TOPIC, topic_name)].result()
        }

    def delete_topic(self, topic_name):
        self.admin_client.delete_topics([topic_name], operation_timeout=30)

    def produce_message(self, topic_name, msg_value):
        self.producer.produce(
            topic = topic_name,
            key = "haha",
            value = msg_value.encode('utf-8'),
            callback = lambda err, msg : print(f"{msg.key().decode('utf-8')} : {msg.value().decode('utf-8')} => {msg.topic()}({msg.partition()})")
        )
    
        self.producer.flush()

    def consumer(self, topic_name, group_id, offset):
        consumer = Consumer({
            'bootstrap.servers': self.bootstrap_servers,
            'group.id': group_id,
            'auto.offset.reset': offset
        })
        consumer.subscribe([topic_name])
        return consumer

    def consume_messages(self, consumer, timeout=1.0):
        try:
            while True:
                msg = consumer.poll(timeout)
                if msg is None:
                    continue
                if msg.error():
                    if msg.error().code() == KafkaError._PARTITION_EOF:
                        print('End of partition reached {0}/{1}'.format(msg.topic(), msg.partition()))
                    elif msg.error():
                        raise KafkaException(msg.error())
                else:
                    print(f"{msg.key().decode('utf-8')}: {msg.value().decode('utf-8')}")
        except KeyboardInterrupt:
            pass
        finally:
            consumer.close()

In [20]:
ck = ConfluentKafka(bootstrap_servers)
_topic = 'sink'
ck.create_topic(_topic, 5, 3)

In [21]:
ck.list_topic()

connect_offsets
demo
my_status_topic
__consumer_offsets
sink
hyunsoo
temp
user
connect_configs


In [17]:
ck.delete_topic(_topic)

In [12]:
ck.describe_topic('demo')

# ck.modify_partition(_topic, 2)

{'partitions': 4,
 'replication_factor': 1,
 'config': {'compression.type': ConfigEntry(compression.type="producer"),
  'leader.replication.throttled.replicas': ConfigEntry(leader.replication.throttled.replicas=""),
  'remote.storage.enable': ConfigEntry(remote.storage.enable="false"),
  'message.downconversion.enable': ConfigEntry(message.downconversion.enable="true"),
  'min.insync.replicas': ConfigEntry(min.insync.replicas="1"),
  'segment.jitter.ms': ConfigEntry(segment.jitter.ms="0"),
  'local.retention.ms': ConfigEntry(local.retention.ms="-2"),
  'cleanup.policy': ConfigEntry(cleanup.policy="delete"),
  'flush.ms': ConfigEntry(flush.ms="9223372036854775807"),
  'follower.replication.throttled.replicas': ConfigEntry(follower.replication.throttled.replicas=""),
  'segment.bytes': ConfigEntry(segment.bytes="1073741824"),
  'retention.ms': ConfigEntry(retention.ms="604800000"),
  'flush.messages': ConfigEntry(flush.messages="9223372036854775807"),
  'message.format.version': ConfigEn

In [6]:
for i in range(10,20):
    ck.produce_message(_topic, str(i))

haha : 10 => yein(4)
haha : 11 => yein(4)
haha : 12 => yein(4)
haha : 13 => yein(4)
haha : 14 => yein(4)
haha : 15 => yein(4)
haha : 16 => yein(4)
haha : 17 => yein(4)
haha : 18 => yein(4)
haha : 19 => yein(4)


In [5]:
cc = confkafka.consumer(_topic, group_id, "earliest")

confkafka.consume_messages(cc)

haha: 10
haha: 10
haha: 11
haha: 12
haha: 13
haha: 14
haha: 15
haha: 16
haha: 17
haha: 18
haha: 19


In [10]:
admin_client.delete_topics(['hyunsoo'], operation_timeout=30)

{'hyunsoo': <Future at 0x75cf22232e80 state=running>}

In [14]:
from confluent_kafka import Producer, Consumer

def create_producer(bootstrap_servers):
    producer = Producer({'bootstrap.servers': bootstrap_servers})
    return producer

producer = create_producer(bootstrap_servers)

In [30]:
for val in range(1,10):
    send_msg = f"NUMBER => {val}"

    producer.produce(
        topic = topic_name,
        key = "haha",
        value = send_msg.encode('utf-8'),
        callback = lambda err, msg : print(f"{msg.key().decode('utf-8')} : {msg.value().decode('utf-8')} => {msg.topic()}({msg.partition()})")
    )

    producer.flush()

haha : NUMBER => 1 => hyunsoo(0)
haha : NUMBER => 2 => hyunsoo(0)
haha : NUMBER => 3 => hyunsoo(0)
haha : NUMBER => 4 => hyunsoo(0)
haha : NUMBER => 5 => hyunsoo(0)
haha : NUMBER => 6 => hyunsoo(0)
haha : NUMBER => 7 => hyunsoo(0)
haha : NUMBER => 8 => hyunsoo(0)
haha : NUMBER => 9 => hyunsoo(0)


In [26]:
def create_consumer(bootstrap_servers, group_id, topics):
    consumer = Consumer({
        'bootstrap.servers': bootstrap_servers,
        'group.id': group_id,
        'auto.offset.reset': 'earliest'
    })
    consumer.subscribe([topics])
    return consumer

consumer = create_consumer(bootstrap_servers, group_id, topic_name)

In [27]:
def consume_messages(consumer, timeout=1.0):
    try:
        while True:
            msg = consumer.poll(timeout)
            if msg is None:
                continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    print('End of partition reached {0}/{1}'.format(msg.topic(), msg.partition()))
                elif msg.error():
                    raise KafkaException(msg.error())
            else:
                print(f"{msg.key().decode('utf-8')}: {msg.value().decode('utf-8')}")
    except KeyboardInterrupt:
        pass
    finally:
        consumer.close()

consume_messages(consumer)

haha: NUMBER => 1
haha: NUMBER => 2
haha: NUMBER => 3
haha: NUMBER => 4
haha: NUMBER => 5
haha: NUMBER => 6
haha: NUMBER => 7
haha: NUMBER => 8
haha: NUMBER => 9
haha: NUMBER => 1
haha: NUMBER => 2
haha: NUMBER => 3
haha: NUMBER => 4
haha: NUMBER => 5
haha: NUMBER => 6
haha: NUMBER => 7
haha: NUMBER => 8
haha: NUMBER => 9
haha: NUMBER => 1
haha: NUMBER => 1
haha: NUMBER => 2
haha: NUMBER => 3
haha: NUMBER => 4
haha: NUMBER => 5
haha: NUMBER => 6
haha: NUMBER => 7
haha: NUMBER => 8
haha: NUMBER => 9
haha: NUMBER => 1
haha: NUMBER => 2
haha: NUMBER => 3
haha: NUMBER => 4
haha: NUMBER => 5
haha: NUMBER => 6
haha: NUMBER => 7
haha: NUMBER => 8
haha: NUMBER => 9
